# Intermediate SAMURAI Tutorial

---

This interactive tutorial will demonstrate how parameter value choices affect the final SAMURAI analysis. SAMURAI is a variational analysis technique that is described in Bell et al. (2012), Foerster et al. (2014), Foerster and Bell (2017), Cha and Bell (2021), and other publications. The SAMURAI analysis yields a maximum likelihood estimate of the atmospheric state for a given set of observations and error estimates by minimizing a variational cost function. 

This tutorial will focus on a few scientific parameters in SAMURAI and how they can affect the smoothing, ... .

---


## Tutorial Overview

### 1. Setup 

#### 1.1 Directory organization 

<div>
<img src="../images/wind_guided_structure.png" width="200"/>
</div>

#### 1.2 QC-ed input data, parameter files, and center files:

**a. Parameter #1: Gaussian filter lengths**
* cfrad.20090605_220449.252_to_20090605_220456.362_DOW6_PPI.nc
* cfrad.20090605_220450.046_to_20090605_220457.156_DOW7_PPI.nc
  
**b. Parameter #2: Spline Cutoffs**
* cfrad.20090605_220449.252_to_20090605_220456.362_DOW6_PPI.nc
* cfrad.20090605_220450.046_to_20090605_220457.156_DOW7_PPI.nc

**c. Parameter #3: ???**
* cfrad.20090605_220449.252_to_20090605_220456.362_DOW6_PPI.nc
* cfrad.20090605_220450.046_to_20090605_220457.156_DOW7_PPI.nc

*The QC process is not included in this tutorial. An example of one type of QC can be found in the QC tutorial and HawkEdit/solo are other tools to QC data.


For this tutorial, all center files are provided. The parameter files are also filled with most parameters. For these exercises, we will be modifying one or two parameters at a time.

#### 1.3 Note on task cells

This notebook uses two colored cells to indicate tasks.

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

These text blocks help the user modify the parameter files or other functions in *external* text files.

</div>

<div class="alert alert-block alert-warning"> <b>Cell Task: run a command.</b> 

These text blocks instruct the users to run a command *in* a cell within the Jupyter notebook.

</div>

#### 1.4 Environment and packages¶

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. 

In [ ]:
import os
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
base_dir = os.environ['BASE_DIR']
!echo "Base directory: " $BASE_DIR

In [ ]:
# import plotting/data packages
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import xarray as xr
import matplotlib as mpl
import cartopy.crs as ccrs
from metpy.plots import ctables
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
mpl.rcParams['figure.dpi'] = 300

We will copy the required data to one directory as SAMURAI requires and organize the directory by performing the following commands:

In [ ]:
# make subdirectory within data for input and output directory
!rm -rf ${BASE_DIR}/data/wind_guided/input
!mkdir -p ${BASE_DIR}/data/wind_guided/input

# create samurai output directory
!rm -rf ${BASE_DIR}/data/wind_guided/output_sam
!mkdir -p ${BASE_DIR}/data/wind_guided/output_sam

# copy raw files to input directory
!cp ${RAW_DIR}/wind_guided/*.nc ${BASE_DIR}/data/wind_guided/input
!cp ${RAW_DIR}/wind_guided/Generate_center.pl ${BASE_DIR}/params/wind_guided


### 2. SAMURAI Parameter #1: Gaussian Filter Lengths 

#### 2.1 Gaussian filter background
This parameter...

These numbers set the Gaussian recursive filter length scale in gridpoints. If set to ‘-1’, the recursive filter is turned off for the given dimension. The minimum recommended value is 2.0, with higher values corresponding to larger spatial influence of the observations. Higher values also result in a smoother analysis.

| Parameter | Description |
| --- | --- |
|i_filter_length, j_filter_length, k_filter_length|Filter lengths for each dimension, where the units are grid points.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

The values we will test in this section are the following:

...


</div>


#### 2.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/intermediate_samurai/samurai_params_


#### 2.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s = base_dir+"/data/wind_guided/output_sam/"
file_s = "samurai_XYZ_analysis.nc"
ds_radar_s = xr.open_dataset(inDir_s+file_s).squeeze()
ds_radar_s.load()

#### Let's use a function to shorten the code.

In [ ]:
def plot_samurai(plotting_lon_s, plotting_lat_s, plotting_var_s, plotting_var_u_s, plotting_var_v_s, ref_cmap, ref_norm, step=4):
    fig = plt.figure(figsize=(12,12))
    ax = plt.axes(projection=ccrs.PlateCarree())
           
    cf1 = ax.pcolormesh( plotting_lon_s, plotting_lat_s, plotting_var_s
                        , cmap=ref_cmap, norm=ref_norm
                        , alpha=0.8, shading='auto'
                        , transform=ccrs.PlateCarree() 
                        )
    cf_q = ax.quiver( plotting_lon_s[::step], plotting_lat_s[::step]
                        , plotting_var_u_s[::step, ::step], plotting_var_v_s[::step, ::step]
                        , scale=1000, width=0.004
                        , color='k'
                        , transform=ccrs.PlateCarree() 
                        )
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
    cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)

In [ ]:
## Set NWS reflectivity colorbar:
ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity', -20, 0.5)
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s = ds_radar_s.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s = ds_radar_s.U.sel(altitude=plotting_alt).data
plotting_var_v_s = ds_radar_s.V.sel(altitude=plotting_alt).data
plotting_lon_s = ds_radar_s.longitude
plotting_lat_s = ds_radar_s.latitude

plot_samurai(plotting_lon_s, plotting_lat_s, plotting_var_s, plotting_var_u_s, plotting_var_v_s, ref_cmap, ref_norm, 4)


### 3. SAMURAI Parameter #2: Spline Cutoffs

#### 3.1 Spline Cutoff background
This parameter...

These numbers set the spline cutoff length scale in gridpoints. See Ooyama (2002) for a discussion of the spline cutoff length. The recommended value is 2.0.

Reference: 

Ooyama, K. V., 2002: The Cubic-Spline Transform Method: Basic Definitions and Tests in a 1D Single Domain. Mon. Wea. Rev., 130, 2392–2415, [link](https://doi.org/10.1175/1520-0493(2002)130<2392:TCSTMB>2.0.CO;2).

| Parameter | Description |
| --- | --- |
|i_spline_cutoff, j_spline_cutoff, k_spline_cutoff|Spline cutoff length scale in grid points.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

The values we will test in this section are the following:

...


</div>


#### 3.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/intermediate_samurai/samurai_params_


#### 3.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s_2 = base_dir+"/data/wind_guided/output_sam/"
file_s_2 = "samurai_XYZ_analysis.nc"
ds_radar_s_2 = xr.open_dataset(inDir_s_2+file_s_2).squeeze()
ds_radar_s_2.load()

#### Plot the output at 1.5 km altitude

In [ ]:
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s_2 = ds_radar_s_2.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s_2 = ds_radar_s_2.U.sel(altitude=plotting_alt).data
plotting_var_v_s_2 = ds_radar_s_2.V.sel(altitude=plotting_alt).data
plotting_lon_s_2 = ds_radar_s_2.longitude
plotting_lat_s_2 = ds_radar_s_2.latitude

plot_samurai(plotting_lon_s_2, plotting_lat_s_2, plotting_var_s_2, plotting_var_u_s_2, plotting_var_v_s_2, ref_cmap_2, ref_norm_2, 4)


### 4. SAMURAI Parameter #3: 

#### 4.1  background
This parameter...


| Parameter | Description |
| --- | --- |
|i_spline_cutoff, j_spline_cutoff, k_spline_cutoff|Spline cutoff length scale in grid points.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

The values we will test in this section are the following:

...


</div>


#### 4.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/intermediate_samurai/samurai_params_


#### 4.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s_3 = base_dir+"/data/wind_guided/output_sam/"
file_s_3 = "samurai_XYZ_analysis.nc"
ds_radar_s_3 = xr.open_dataset(inDir_s_3+file_s_3).squeeze()
ds_radar_s_3.load()

#### Plot the output at 1.5 km altitude

In [ ]:
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s_3 = ds_radar_s_3.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s_3 = ds_radar_s_3.U.sel(altitude=plotting_alt).data
plotting_var_v_s_3 = ds_radar_s_3.V.sel(altitude=plotting_alt).data
plotting_lon_s_3 = ds_radar_s_3.longitude
plotting_lat_s_3 = ds_radar_s_3.latitude

plot_samurai(plotting_lon_s_3, plotting_lat_s_3, plotting_var_s_3, plotting_var_u_s_3, plotting_var_v_s_3, ref_cmap_3, ref_norm_3, 4)
